In [127]:
import numpy as np 
import pandas as pd

# Loading the multiple choices dataset, we will not look to the free form data on this study
mc = pd.read_csv('data/multipleChoiceResponses.csv', low_memory=False)

# Separating questions from answers
# This Series stores all questions
mcQ = mc.iloc[0,:]
# This DataFrame stores all answers
mcA = mc.iloc[1:,:]

In [128]:
# removing everyone that took less than 4 minutes or more than 600 minutes to answer the survey
less3 = mcA[round(mcA.iloc[:,0].astype(int) / 60) <= 4].index
mcA = mcA.drop(less3, axis=0)
more300 = mcA[round(mcA.iloc[:,0].astype(int) / 60) >= 600].index
mcA = mcA.drop(more300, axis=0)

# removing gender trolls, because we noticed from other kernels thata there are some ouliers here
gender_trolls = mcA[(mcA.Q1 == 'Prefer to self-describe') | (mcA.Q1 == 'Prefer not to say')].index
mcA = mcA.drop(list(gender_trolls), axis=0)

# removing student trolls, because a student won't make more than 250k a year.
student_trolls = mcA[((mcA.Q6 == 'Student') & (mcA.Q9 > '500,000+')) | \
                     ((mcA.Q6 == 'Student') & (mcA.Q9 > '400-500,000')) | \
                     ((mcA.Q6 == 'Student') & (mcA.Q9 > '300-400,000')) | \
                     ((mcA.Q6 == 'Student') & (mcA.Q9 > '250-300,000'))].index
mcA = mcA.drop(list(student_trolls), axis=0)

# dropping all NaN and I do not wish to disclose my approximate yearly compensation, because we are only interested in respondents that revealed their earnings
mcA = mcA[~mcA.Q9.isnull()].copy()
not_disclosed = mcA[mcA.Q9 == 'I do not wish to disclose my approximate yearly compensation'].index
mcA = mcA.drop(list(not_disclosed), axis=0)

In [129]:
# Creating a table with personal data
personal_data = mcA.iloc[:,:13].copy()

# renaming columns
cols = ['survey_duration', 'gender', 'gender_text', 'age', 'country', 'education_level', 'undergrad_major', 'role', 'role_text',
        'employer_industry', 'employer_industry_text', 'years_experience', 'yearly_compensation']
personal_data.columns = cols

# Drop text and survey_duration columns 
personal_data.drop(['survey_duration', 'education_level', 'undergrad_major', 'gender',
                    'gender_text', 'role_text', 'employer_industry_text'], axis=1, inplace=True)

personal_data = pd.concat([personal_data, mcA.loc[:, [c for c in mcA.columns if c.startswith('Q11_Part')]]], axis=1)

personal_data.head(3)

,age,country,role,employer_industry,years_experience,yearly_compensation,Q11_Part_1,Q11_Part_2,Q11_Part_3,Q11_Part_4,Q11_Part_5,Q11_Part_6,Q11_Part_7
2,30-34,Indonesia,Other,Manufacturing/Fabrication,5-10,"10-20,000",NaN,NaN,NaN,NaN,NaN,None of these activities are an important part...,NaN
3,30-34,United States of America,Data Scientist,I am a student,0-1,"0-10,000",Analyze and understand data to influence produ...,NaN,NaN,NaN,NaN,NaN,NaN
5,22-24,India,Data Analyst,I am a student,0-1,"0-10,000",NaN,NaN,NaN,NaN,NaN,NaN,Other


In [131]:
from pandas.api.types import CategoricalDtype

# transforming compensation into category type and ordening the values
categ = ['0-10,000', '10-20,000', '20-30,000', '30-40,000', '40-50,000',
         '50-60,000', '60-70,000', '70-80,000', '80-90,000', '90-100,000',
         '100-125,000', '125-150,000', '150-200,000', '200-250,000', '>250,000']

too_big_salary = ['250-300,000', '300-400,000', '400-500,000', '500,000+']
personal_data.yearly_compensation = personal_data.yearly_compensation.apply(lambda x: x if x not in too_big_salary else '>250,000')
cat_type = CategoricalDtype(categories=categ, ordered=True)
personal_data.yearly_compensation = personal_data.yearly_compensation.astype(cat_type)
# Doing this we are transforming the category "I do not wish to disclose my approximate yearly compensation" into NaN

# transforming age into category type and sorting the values
categ = ['18-21', '22-24', '25-29', '30-34', '35-39', '40-44', 
         '45-49', '50-54', '55-59', '60-69', '70-79', '80+']
cat_type = CategoricalDtype(categories=categ, ordered=True)
personal_data.age = personal_data.age.astype(cat_type)

# transforming years of experience into category type and sorting the values
categ = ['0-1', '1-2', '2-3', '3-4', '4-5', '5-10',
         '10-15', '15-20', '20-25', '25-30', '30+']
cat_type = CategoricalDtype(categories=categ, ordered=True)
personal_data.years_experience = personal_data.years_experience.astype(cat_type)

In [135]:
df = personal_data.copy()

In [136]:
from pandas.api.types import CategoricalDtype
categ = ['18-21', '22-24', '25-29', '30-34', '35-39', '40-44', 
         '45-49', '50-54', '55-59', '60-69', '70-79', '80+']
cat_type = CategoricalDtype(categories=categ, ordered=True)
df.age = df.age.astype(cat_type)

In [137]:
df.age[df.age > '60-69'] = '60-69'
df.age = df.age.astype(object)
df.age[df.age == '60-69'] = '60+'

/home/ptyshevskyi/envs/loc_env/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [138]:
special_countries = ['Australia', 'Brazil', 'Canada', 'China', 'France', 'Germany',
                     'India', 'Italy', 'Japan', 'Mexico', 'Russia', 'Spain', 'United Kingdom of Great Britain and Northern Ireland',
                     'United States of America']
df.country = df.country.apply(lambda x: x if x in special_countries else 'Other')

In [139]:
top_industries = ['Academics/Education', 'Accounting/Finance', 'Computers/Technology',
                  'Government/Public Service', 'Insurance/Risk Assessment', 
                  'Medical/Pharmaceutical', 'Online Service/Internet-based Services']
df.employer_industry = df.employer_industry.apply(lambda x: x if x in top_industries else 'Other')
df.employer_industry.value_counts()

Other                                     4761
Computers/Technology                      3568
Academics/Education                       1630
Accounting/Finance                        1012
Online Service/Internet-based Services     611
Medical/Pharmaceutical                     521
Government/Public Service                  448
Insurance/Risk Assessment                  405
Name: employer_industry, dtype: int64

In [140]:
top_roles = ['Business Analyst', 'Consultant', 'Data Analyst', 'Data Engineer', 'Data Scientist',
             'Research Scientist', 'Software Engineer', 'Student']
df.role = df.role.apply(lambda x: x if x in top_roles else 'Other')

In [141]:
df.years_experience = df.years_experience.apply(lambda x: x if x not in ['15-20', '20-25', '25-30'] else '15+')

In [142]:
df.Q11_Part_1 = df.Q11_Part_1.apply(lambda x: 0 if pd.isnull(x) else 1)
df.Q11_Part_2 = df.Q11_Part_2.apply(lambda x: 0 if pd.isnull(x) else 1)
df.Q11_Part_3 = df.Q11_Part_3.apply(lambda x: 0 if pd.isnull(x) else 1)
df.Q11_Part_4 = df.Q11_Part_4.apply(lambda x: 0 if pd.isnull(x) else 1)
df.Q11_Part_5 = df.Q11_Part_5.apply(lambda x: 0 if pd.isnull(x) else 1)

In [143]:
part6_combined = df.Q11_Part_6.copy()
for i, row in df.iterrows():
    isna = pd.isnull(row['Q11_Part_6']) & pd.isnull(row['Q11_Part_7'])
    part6_combined[i] = 0 if isna else 1

df = df.drop(['Q11_Part_7'], axis=1)
df.Q11_Part_6 = part6_combined

In [144]:
df.to_csv('salary_prediction.csv')